In [120]:
import os
from sklearn.decomposition import PCA
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler
from statsmodels.stats.multitest import multipletests
from scipy.stats import ttest_ind_from_stats
from scipy.stats import f_oneway as anova
import scipy
import plotly.graph_objs as go
from plotly.offline import plot
import json
import plotly.io as pio
from plotly.offline import iplot
import plotly as py
from matplotlib_venn import venn2, venn3
import matplotlib.pyplot as plt
import matplotlib
import plotly.colors
import plotly.express as px
from plotly.subplots import make_subplots
from scipy.stats import t
import seaborn as sns
from datetime import datetime, timedelta
import plotly.express as px
from pathlib import Path
import statsmodels.api as sm 
from statsmodels.formula.api import ols 
  
import math
from select import select
import numpy as np
import pandas as pd
from SCP_plotter import *
from SCP_processor import *
# import requests
# from requests.auth import HTTPBasicAuth

#only used for the app

# import django
# from django.conf import settings
# from django.contrib.auth.decorators import login_required, permission_required
# from file_manager.models import DataAnalysisQueue, SampleRecord, \
#     SavedVisualization, VisualizationApp, UserSettings, ProcessingApp
# from django.shortcuts import render
# from django.conf import settings


In [150]:
#Method put files in, one dictionary for each analysis, if there is only one file or you are looking in all files, then you don't need @'s in your SETTINGS_FILE
WRITE_OUTPUT = True
USE_MaxLFQ = False
filelist = [
             {"input1":"input/FP_20/tmt-report/abundance_protein_MD.tsv",
             "input2":"input/FP_20/tmt-report/abundance_peptide_MD.tsv",
             "input3":"report.pg_matrix.tsv",
             "input4":"report.pr_matrix.tsv",
             "input5":"Input/Payne2/Payne_organoids2.0_InputFiles.txt",
             "process_app": "FragPipe"},
          
            ]

            #Order of files
                #For FragPipe
                    #input1= Protein with MBR
                    #input2= Peptide with MBR
                #For DIANN
                    #input1= pg_matrix
                    #input2= pr_matrix 
                    #input3= pg_matrix MS2
                    #input4= pr_matrix MS2
                #For PD
                    #input1= Protein 
                    #input2= Peptide Groups
                    #input3= blank
                    #input4= blank
                    #input5= inpit_files


SETTINGS_FILE = "settings_test_TMT.txt" #tab delimited
    #Format
    #Required Columns
        #Group Name	
        #filter_in: a string pattern contained in desired raw filenames in analysis followed by an @ + the file number (filelist index) to look in
        #filter_out: a string pattern contained in undesired raw filenames
    #Extra columns
        #can be used to group by color, layer, or x position in CV and ID graphs

In [151]:
# define group names and assign analysis to different groups(web app version was done through GUI so
# steps are taken to make sure they have same output)
x = read_files(grouped_input_files=filelist)
data_obj = outer_join_data_objects(x)

settings_table = pd.read_table(SETTINGS_FILE,sep="\t")
saved_settings = settings_table.set_index("Conditions").to_dict(orient="index")
#any run with any of the filter_out items will not be used.

for eachGroup in saved_settings:
    i = 0
    saved_settings[eachGroup]["records"] = []
    filterOutType = type(saved_settings[eachGroup]["filter_out"])
    if filterOutType == str or filterOutType == int or filterOutType == float and not pd.isna(saved_settings[eachGroup]["filter_out"]):
        filterOut = str.split(saved_settings[eachGroup]["filter_out"],sep = ",")
    else:
        filterOut = ["M@di"]
    if len(str.split(str(saved_settings[eachGroup]["filter_in"]),sep = "@")) > 1: #multiple files, only some have the runs for this group
        user_list = []
        #add all runs from all analyses to be probed
        for each_fileID in str.split(str(saved_settings[eachGroup]["filter_in"]),sep = "@")[1:]:
            for eachIdentifier in data_obj["run_metadata"]["Run Identifier"]:    
                currentRun = data_obj["run_metadata"][data_obj["run_metadata"]["Run Identifier"] == eachIdentifier]["Run Names"] 
                if currentRun.size == 1:
                    if each_fileID == str.split(eachIdentifier,sep="-")[0] and list(currentRun)[0] not in user_list:
                        user_list.append(list(currentRun)[0])
                else:
                    pass
        #filter for runs that relate to this gorup within those analyses
        for run_name in user_list:
            if str.split(str(saved_settings[eachGroup]["filter_in"]),sep = "@")[0] in run_name and (not any(item in run_name for item in filterOut)):
                saved_settings[eachGroup]["records"].append(list(data_obj["run_metadata"][data_obj["run_metadata"]["Run Names"] == run_name]["Run Names"])[0]) 
            else:
                pass
    elif len(str.split(str(saved_settings[eachGroup]["filter_in"]),sep = "@")) == 1: #across all files or maybe there is only one
            for run_name in data_obj["run_metadata"]["Run Names"]:
                if str(saved_settings[eachGroup]["filter_in"]) in run_name and (not any(item in run_name for item in filterOut)):
                    saved_settings[eachGroup]["records"].append(data_obj["run_metadata"]["Run Names"][i])  
                i = i + 1   

#add the order of each column
ignore_columns = ["filter_in","filter_out"]
category_columns = [x for x in settings_table.columns.to_list() if x not in ignore_columns]

for eachCol in category_columns:
    saved_settings["Order@"+eachCol] = settings_table[eachCol].drop_duplicates().to_list()
    

***


In [152]:
#Missing val plot plot(protein and peptides)
# print(saved_settings)
plot_options={
            "mean label": "True",
            "error bar": "ci95", #None, stdev, or ci95
            "X Title": "Conditions",
            "cutoffs": [33.3,50,99],
            "Y Title": "Protein Identifications",
            "color": ["#3E6990", "#7aa8e6","#AABD8C", "#E9E3B4", "#C6878F", "#fac8d3","#F39B6D",  "pink",
                  "orange", "brown", "pink", "gray", "olive", "cyan", "black","red",
                  "yellow","green","blue","indigo","violet"],
            "width": 700,
            "height": 450,
            "font": dict(size=32, family="Arial black"),
            "ID mode": "stacked",#stacked
            "Group By X": "Conditions", #ID_Mode does MS2 vs MBR
        }
plot_options["plot_type"] = "1" # 1 is protein, 2 is peptide
figure ,_ ,_ =Missing_Values_Plots(data_obj, plot_options, saved_settings)
figure.write_image("greedy_missing_val.png", scale=10, width = 1000, height=1000)
figure.show()

['13ng_inclusion_90s_5ppm', '13ng_inclusion_90s_5ppm.2', '13ng_inclusion_90s_5ppm.3', '13ng_inclusion_90s_5ppm.4', '13ng_inclusion_90s_5ppm.5', 'Annotated Sequence', 'Symbol']
['13ng_standard', '13ng_standard.1', '13ng_standard.3', '13ng_standard.4', '13ng_standard.5', '13ng_standard.7', 'Annotated Sequence', 'Symbol']
['library', 'library.1', 'library.2', 'library.3', 'library.4', 'library.5', 'library.6', 'library.7', 'library.8', 'Annotated Sequence', 'Symbol']
['134N-4ng', '134N-4ng.1', '134N-4ng.2', 'Annotated Sequence', 'Symbol']
Index(['Symbol', '0-17', '0-19', '0-20', '0-21', '0-22'], dtype='object')
      Missing Values Rate    Symbol
0                     0.0    GATD3B
1                    80.0   ZC3H11B
2                   100.0     MCTS2
3                   100.0      UBA6
4                   100.0     ESYT2
...                   ...       ...
2244                 40.0    MAD1L1
2245                100.0  DYNC1LI1
2246                100.0     IKBKG
2247                 20.

C:\Users\RTKLAB\AppData\Local\Temp\ipykernel_22772\2447071490.py:89: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\RTKLAB\AppData\Local\Temp\ipykernel_22772\2447071490.py:89: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\RTKLAB\AppData\Local\Temp\ipykernel_22772\2447071490.py:89: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\RTKLAB\AppData\Local\Temp\ipykernel_22772\2447071490.py:89: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\RTKLAB\AppData\Local\Temp\ipykernel_22772\2447071490.py:89: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pand

In [158]:
#ID plot(protein and peptides)
plot_options={
            "mean label": "True",
            "error bar": "ci95", #None, stdev, or ci95
            "X Title": "Conditions",
            "Y Title": "Protein Identifications",
            "color": ["#3E6990", "#7aa8e6","#AABD8C", "#E9E3B4", "#C6878F", "#fac8d3","#F39B6D",  "pink",
                  "orange", "brown", "pink", "gray", "olive", "cyan", "black","red",
                  "yellow","green","blue","indigo","violet"],
            "width": 700,
            "height": 450,
            "font": dict(size=32, family="Arial black"),
            "ID mode": "MS2",#grouped, total, MS2, grouped_stacked, stacked
            "Group By X": "Gradient Length", #ID_Mode does MS2 vs MBR
            "Group By Color": "Acquisition Mode",#ID_Mode does MS2 vs MBR
            "Group By Stack": "ID_Mode",#ID_Mode does MS2 vs MBR
        }
plot_options["plot_type"] = "1" # 1 is protein, 2 is peptide
figure ,_ ,_ =ID_plots(data_obj, plot_options, saved_settings)
figure.write_image("asdf.png", scale=10, width = 1000, height=1000)
figure.show()

In [154]:
# CV violin plot
plot_options={    
        "median label": "True", #only works for total and MS2, can be found in CV_summary if you set WRITE_OUTPUT = True
        "box": False,
        "X Title": "Conditions (protein)",
        "Y Title": "CV of Abundance (%)",
        "color": ["#3E6990", "#7aa8e6","#C6878F", "#fac8d3","#AABD8C", "#E9E3B4",  "#F39B6D", "#C6878F", "pink",
                  "orange", "brown", "pink", "gray", "olive", "cyan", "black","red",
                  "yellow","green","blue","indigo","violet"],
        "width": 700,
        "height": 450,
        "font": dict(size=16, family="Arial black"),
        "ylimits": [-10, 400],
        "CV mode": "grouped_stacked",#grouped, total, MS2, grouped_stacked, stacked
        "Group By X": "Gradient Length", #ID_Mode does MS2 vs MBR
        "Group By Color": "Acquisition Mode",#ID_Mode does MS2 vs MBR
        "Group By Stack": "ID_Mode" #ID_Mode does MS2 vs MBR
    }
plot_options["plot_type"] = 1 # 1 is protein, 2 is peptide
figure, _, _ =CV_plots(data_obj, plot_options, saved_settings)
figure.show()


MS2 CVs...


In [155]:
# ID Venns plot
plot_options={
            "compare groups": ["4ng Greedy","13ng Greedy"],
            "inclusion list": "proteins_symbols.csv",
            "title": "Venn Diagram (Protein)",
            "opacity": 0.75,
            "color": ["#3E6990", "#C6878F", "#AABD8C","#fac8d3","#7aa8e6", "#E9E3B4","#00FF00", "#FFFF00", "#FF0000", "yellow", "red",
                      "green", "purple", "orange", "brown", "pink",
                      "gray",  "olive", "cyan", "blue",  "black", ]
        }
plot_options["plot_type"] = 1 # 1 is protein, 2 is peptide
figure, _, _ =inclusion_venn_plots(data_obj, plot_options, saved_settings)
figure.show()



In [156]:
# ID Venns plot
plot_options={
            "compare groups": ["Library","4ng Greedy","13ng Greedy"],
            "title": "Venn Diagram (Protein)",
            "opacity": 0.75,
            "color": ["#3E6990", "#C6878F", "#AABD8C","#fac8d3","#7aa8e6", "#E9E3B4","#00FF00", "#FFFF00", "#FF0000", "yellow", "red",
                      "green", "purple", "orange", "brown", "pink",
                      "gray",  "olive", "cyan", "blue",  "black", ]
        }
plot_options["plot_type"] = 1 # 1 is protein, 2 is peptide
figure, _, _ =venns_plots(data_obj, plot_options, saved_settings)
figure.show()



In [157]:
# Protein Ranked Abundance plot
plot_options={
            "reference_group": "Library",
            "compare groups": ["13ng Greedy", "4ng Greedy", "13ng Standard"],
            "title": "Ranked Abundance",
            "color": ["blue", "red", "green", "black", "yellow", "purple",
                      "orange", "brown", "pink", "gray", "olive", "cyan"],
            "symbol": ['circle', 'circle', 'circle'],
            "marker_size": 8,
            "width": 700,
            "height": 450,
            "font": dict(size=16, family="Arial black"),
            "ylimits": [],
            "xlimits": [],
            "X Title": "Ranked Abundance",
            "Y Title": "Average Reporter Ion Abundance",
        }
figure, _, _ =Rank_Abundance_Plots(data_obj, plot_options, saved_settings)
figure.show()

C:\Users\RTKLAB\AppData\Local\Temp\ipykernel_22772\1853674399.py:11: FutureWarning:

Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.

C:\Users\RTKLAB\AppData\Local\Temp\ipykernel_22772\1853674399.py:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



KeyError: '13ng Standard'

In [ ]:
# Protein Abundance Volcano plot
plot_options={
            "compare groups": ["4ng Greedy","13ng Greedy"],
            "title": "Volcano Plot",
            "X Title": "Log2 Fold Change",
            "Y Title": "-Log10(adjusted p-value)",
            "up color": "red",
            "down color": "blue",
            "all color": "gray",
            "width": 700,
            "height": 450,
            "font": dict(size=16, family="Arial black"),
            "ylimits": [],
            "xlimits": []
        }
figure, _, _ =volcano_plots(data_obj, plot_options, saved_settings)
figure.show()


NameError: name 'volcano_plots' is not defined

In [ ]:
# Protein Abundance PCA plot
plot_options={
            "compare groups": ["4ng Greedy","13ng Greedy","Library", "13ng Standard"],
            "title": "PCA Analysis",
            "color": ["blue", "red", "green", "black", "yellow", "purple",
                      "orange", "brown", "pink", "gray", "olive", "cyan"],
            "symbol": ['circle', 'circle', 'circle'],
            "marker_size": 8,
            "width": 700,
            "height": 450,
            "font": dict(size=16, family="Arial black"),
            "ylimits": [],
            "xlimits": []
        }
figure, _, _ =PCA_plots(data_obj, plot_options, saved_settings)
figure.show()


In [ ]:
# Protein Abundance Heat Map
plot_options={
            "compare groups":  ["4ng Greedy","13ng Greedy","Library"],
            "log2_transform": False,
            "significant_only": True,
            "alpha": 0.05, 
            "min_fold_change": 2
        }
figure, _, _ =heatmap_plots(data_obj, plot_options, saved_settings)
figure.show()